# Cleaning scrapped data

## Libraries

In [1]:
import pandas as pd
import numpy as np

## Processing

In [2]:
df = pd.read_csv('glassdoor_jobs.csv',header = 0, index_col=0)

In [3]:
# Format corrections

df['Company Name'] = df['Company Name'].astype('string')

In [4]:
# Marking the entries where per hour estimate is provided

df['hourly flag'] = df['Salary Estimate'].apply(lambda x: 1 if 'hour' in x.lower() else 0)

In [5]:
# Marking entries where Employer provided estaimte is available

df['employer provided flag'] = df['Salary Estimate'].apply(lambda x: 1 if 'employer' in x.lower() else 0)

In [6]:
# Remove the entries with salary not found

df = df[df['Salary Estimate'] != '-1']

Add Min and Max ranges of salary to the dataframe

In [7]:
salary = df['Salary Estimate'].apply(lambda x: x.split("(")[0])
# Split will return all the splits created in the string based on delimiter provided. 
# Since we only need the salary, we pick the first split element from the output array in lambda function

# Replace $ and K in salary to get numeric range

salary = salary.apply(lambda x: x.replace("K","").replace("$",""))

salary = salary.apply(lambda x: x.lower().replace("per hour","").replace("employer provided salary:",""))

# Put range of salary in dataframe

df['min_salary'] = salary.apply(lambda x: x.split("-")[0]).astype('int8')
df['max_salary'] = salary.apply(lambda x: x.split("-")[1]).astype('int8')

Company name contains the rating for the company if available. We need to parse to retain just names

In [8]:
df.loc[df['Rating']<0,'company_txt'] = df['Company Name']

df.loc[df['Rating']>=0,'company_txt'] = df['Company Name'].apply(lambda x: x[:-4])

#display(df)

In [17]:
# Extract state from location

df['job_state'] = df['Location'].apply(lambda x: x.split(",")[1] if x != "-1" else "-1")
df['hq_state'] = df['Headquarters'].apply(lambda x: x.split(",")[1] if x != "-1" else "-1")
#display(df)

# Make flag for if job is in HQ location
df['job_in_HQ_flag'] = df['job_state'] == df['hq_state']

In [20]:
# Derive age of company based on current date

today_year = 2021

df['age_company'] = df['Founded'].apply(lambda x: today_year - x if x >=0 else x)

Extract the tool requirement from job descriptions in data

Based on quick search, we find that below are top tools used in industry. We createflags for these

1. SAS
2. Spark
3. Python
4. Matlab
5. Tensorflow
6. R - hard to create a flag for this. Maybe find Caps "R" in JD
7. Tableu

In [43]:
# We process R flag seperately
skill_array = ['SAS','Spark','Python','Matlab','Tensorflow','Tableau','AWS','hadoop']

for skill in skill_array:
    skill = skill.lower()
    df[f"{skill}_flag"] = df['Job Description'].apply(lambda x: 1 if skill in x.lower() else 0)

df["r_flag"] = df['Job Description'].apply(lambda x: 1 if ('r-studio' in x.lower() or 'r studio' in x.lower()) else 0)


In [45]:
#display(df)

# Save progress to csv file
# Can also use pickle to save session for fast load and to prevent losing formatting

df.to_pickle('./cleaned_jobs.pk1')